In [1]:
from kernels import * 
from learning_models import *
from tools import *
import pandas as pd
import numpy as np
from time import time 
from sklearn.svm import SVC
from autoreload import superreload

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
dataX = pd.read_csv("Data/Train/Xtr0.csv", header=None)

In [4]:
dataX.head()

,0
0,TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGG...
1,CGGAGTGATTCAGCAAGAAGGCTCAGACCATTTACACTTCTCCTCA...
2,GCCTCAGGCACTCTCCAGATGGTTAAGATATGGTGTAAGTGTATGT...
3,TCAGTCGCCTGGCCTTCCAGCCTCACAGGCAAGATAAGCCACCCTC...
4,CTTAATGGAAAAATTAGATTATGGGTTGGCTTACCTGGATTTACCT...


In [5]:
data = dataX[0].values.tolist()

In [6]:
data[:10]

['TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGGCGCCCTCTAGAGTTGTGTAAAGAAGTGGCGTCACCTCATTATAAATAAAAGGTTG',
 'CGGAGTGATTCAGCAAGAAGGCTCAGACCATTTACACTTCTCCTCACAGGGCAAAGACTTCTCAGAGACAACTAATATGGCCTAGTGCCTAGTCATTGGAC',
 'GCCTCAGGCACTCTCCAGATGGTTAAGATATGGTGTAAGTGTATGTGCGTGTCTGGCTGCAGTTGATAGTTGGGCAAAGAGGAAACTATCACAAAGAGGAA',
 'TCAGTCGCCTGGCCTTCCAGCCTCACAGGCAAGATAAGCCACCCTCACAGACTTCTCTGTGCTCCCATCTAAACATCTAAATCAGCCATCTGGTTTCCTTA',
 'CTTAATGGAAAAATTAGATTATGGGTTGGCTTACCTGGATTTACCTAGAGTGTCTGCCATGTCTATCAATTGGATTACGGGGCTTTTATTGATTAGGGGGA',
 'GAAACAAGCTAATATAAGGTGGGGGCCTGAAAATTTTATCCAATGGTCATTTAAGAATCGCAATCAATGTCCTCTAGATATATAATTCACCTAAATACGGA',
 'CTTTCTTATCTGTGTGGCTGTGGGCGTGTCTTTGGCAAGCCCCACTGTGAGAGTTCCCTTATCTATGCCTGCAGGCTGTTCTTTTATTTGAAAGAATTCAG',
 'TGATTACTGCATGGAAAAGGGGCCAGGGCAATTACTAGAGGGCTCAATCCCAGTAAGGGGATGTGTACCAGGGAGGGTTGCTTTGACAGGAGATCAGTCTA',
 'GAGATTCTTAACATGCAACCACAAATCGATGCAGTTATGGATCATGTTGAATGGCAACTCACCTGGCAGTGGAGAGCAGAGTTGTTGGCAATCACATATGA',
 'GGGGACTTCTTGAACTGAGCTCCTAGAGCAGCTGGGGTTGTGAT

In [7]:
d = []
for i in range(len(data)):
    s = data[i].replace("A", "0")
    s = s.replace("T", "1")
    s = s.replace("C", "2")
    s = s.replace("G", "3")
    d.append(list(map(int, s)))
data = d

In [8]:
del dataX

In [ ]:
trie = MismatchTrie(verbose=0)
print("-------- Computing mismatch kernel --------")
start = time()
kern = trie.traverse(data, 4, 5, 1)[0]
#kern = normalize_kernel(kern)
end = time()
print("-------- Done in {:.3f} minutes --------".format((end-start)/60))

-------- Computing mismatch kernel --------


In [ ]:
kern.shape

In [33]:
np.array(data).shape

(2000, 101)

In [34]:
dataY = pd.read_csv("Data/Train/Ytr0.csv", index_col=0)
y = dataY['Bound'].values
X = np.array(data)
K = kern

In [35]:
kern = normalize_kernel(kern)

In [39]:
X_train, X_test, y_train, y_test, K_train, K_test = train_test_split(X,y,kern,test_size=0.2)

Total number of examples : 2000
Ratio of positive samples : 0.50
Ratio of negative to positive labels in the data : 1.00


In [40]:
res = []
for lmdb in [0.001,0.01,0.05,0.1,1,10,100,1000]:
    model = SVM(lmbd=lmdb)
    model.train(K_train, y_train)
    y_train_pred = model.predict(K_train)
    y_test_pred = model.predict(K_test)
    train_score = accuracy_score(y_train, y_train_pred)
    test_score = accuracy_score(y_test, y_test_pred)
    print("\n\n------ Lambda = {} ------".format(lmdb))
    print('Train accuracy score = {:.3f}'.format(train_score))
    print('Test accuracy score = {:.3f}\n\n'.format(test_score))
    res.append([lmdb,test_score])


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  3e+03  6e+01  1e+00
 1: -9.1577e-01 -1.7671e+00  4e+01  6e-01  1e-02
 2: -6.4773e-01 -1.6984e+00  1e+00  2e-03  4e-05
 3: -7.9352e-01 -9.3109e-01  1e-01  3e-04  5e-06
 4: -8.4296e-01 -9.1046e-01  7e-02  1e-04  2e-06
 5: -8.6824e-01 -8.9833e-01  3e-02  5e-05  1e-06
 6: -8.7763e-01 -8.9399e-01  2e-02  3e-05  5e-07
 7: -8.8438e-01 -8.9085e-01  7e-03  6e-06  1e-07
 8: -8.8767e-01 -8.8849e-01  8e-04  4e-07  7e-09
 9: -8.8807e-01 -8.8814e-01  8e-05  1e-08  2e-10
10: -8.8810e-01 -8.8811e-01  4e-06  5e-10  9e-12
11: -8.8811e-01 -8.8811e-01  2e-07  2e-11  4e-13
Optimal solution found.


------ Lambda = 0.001 ------
Train accuracy score = 0.618
Test accuracy score = 0.545



			######## y has been reformated to {-1,1} ########

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  3e+03  6e+01  1e+00
 1: -1.0184e+00 -1.8694e+00  5e+01  6e-01  1e-02
 2: -7.7677e-01 -1.7660e+00  4e+00  4e-02  7e-04


In [41]:
best_pred = sorted(res, key = lambda x:x[1])[-1]
print(best_pred)

[1000, 0.5475]


In [16]:
def transform(y, threshold=0.5):
    res = np.zeros(len(y))
    ones = y >= threshold
    res[ones] = 1
    res[~ones] = 0
    return res

In [26]:
res = []
for lmdb in [1e-5, 1e-4, 0.001,0.01,0.05]:
    print("------ Lambda = {} ------".format(lmdb))
    model = KRR(lmbd=lmdb)
    model.train(K_train, y_train)
    y_train_pred = model.predict(K_train)
    y_test_pred = model.predict(K_test)
    for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
        print('\t\tthreshold = {}'.format(threshold))
        res_train = transform(y_train_pred, threshold=threshold)
        res_test = transform(y_test_pred, threshold=threshold)
        train_score = accuracy_score(y_train, res_train)
        test_score = accuracy_score(y_test, res_test)
        print('Train accuracy score = {:.3f}'.format(train_score))
        print('Test accuracy score = {:.3f}\n'.format(test_score))
        res.append([lmdb, threshold, test_score])

------ Lambda = 1e-05 ------
		threshold = 0.1
Train accuracy score = 0.514
Test accuracy score = 0.497

		threshold = 0.2
Train accuracy score = 0.526
Test accuracy score = 0.515

		threshold = 0.3
Train accuracy score = 0.544
Test accuracy score = 0.530

		threshold = 0.4
Train accuracy score = 0.568
Test accuracy score = 0.535

		threshold = 0.5
Train accuracy score = 0.589
Test accuracy score = 0.540

		threshold = 0.6
Train accuracy score = 0.569
Test accuracy score = 0.510

		threshold = 0.7
Train accuracy score = 0.538
Test accuracy score = 0.490

		threshold = 0.8
Train accuracy score = 0.524
Test accuracy score = 0.490

------ Lambda = 0.0001 ------
		threshold = 0.1
Train accuracy score = 0.514
Test accuracy score = 0.495

		threshold = 0.2
Train accuracy score = 0.525
Test accuracy score = 0.515

		threshold = 0.3
Train accuracy score = 0.543
Test accuracy score = 0.530

		threshold = 0.4
Train accuracy score = 0.568
Test accuracy score = 0.535

		threshold = 0.5
Train accur

In [30]:
best_pred = sorted(res, key = lambda x:x[2])[-1]
print(best_pred)

[0.05, 0.4, 0.5525]
